---
### Random Search Hyper Parameter Tuning with YOLOv8n

In [12]:
#YOLOv8n

from ultralytics import YOLO
import random
import csv
import os
import time

# Parameter ranges for YOLOv8n
y8n_param_ranges = {
    'lr0': (0.0005, 0.02),
    'batch': [16, 32, 64],
    'weight_decay': (0.0001, 0.001),
    'momentum': (0.85, 0.98),
    'warmup_epochs': [2, 3, 5, 7],
    'patience': [10, 20, 30]
}

y8n_num_trials = 5
y8n_best_map = 0
y8n_best_params = {}
y8n_csv_filename = "yolov8n_random_search_results.csv"
y8n_results = []

if os.path.exists(y8n_csv_filename):
    os.remove(y8n_csv_filename)

for trial in range(y8n_num_trials):
    random.seed(time.time() + trial)

    y8n_params = {
        'lr0': round(random.uniform(*y8n_param_ranges['lr0']), 5),
        'batch': random.choice(y8n_param_ranges['batch']),
        'weight_decay': round(random.uniform(*y8n_param_ranges['weight_decay']), 5),
        'momentum': round(random.uniform(*y8n_param_ranges['momentum']), 3),
        'warmup_epochs': random.choice(y8n_param_ranges['warmup_epochs']),
        'patience': random.choice(y8n_param_ranges['patience'])
    }

    print(f"\n🔍 YOLOv8n Trial {trial+1}/{y8n_num_trials} | Params: {y8n_params}")

    start_time = time.time()

    y8n_model = YOLO("yolov8n.pt")
    y8n_model.train(
        data="data.yaml",
        epochs=15,
        imgsz=256,
        **y8n_params,
        name=f"trial_{trial+1}",
        project="runs/yolov8n_random_search",
    )

    y8n_metrics = y8n_model.val()
    current_map50 = y8n_metrics.box.map50
    duration_min = round((time.time() - start_time) / 60, 2)

    print(f"\n📊 Trial {trial+1} Validation Metrics:")
    print(f"mAP@0.5:0.95    {y8n_metrics.box.map:.4f}")
    print(f"mAP@0.5:        {y8n_metrics.box.map50:.4f}")
    print(f"mAP@0.75:       {y8n_metrics.box.map75:.4f}")
    print(f"Precision:      {y8n_metrics.box.p.mean():.4f}")
    print(f"Recall:         {y8n_metrics.box.r.mean():.4f}")
    print(f"⏱️ Duration:     {duration_min:.2f} min")

    y8n_result = {
        'trial': trial + 1,
        **y8n_params,
        'map50': round(y8n_metrics.box.map50, 4),
        'map5095': round(y8n_metrics.box.map, 4),
        'map75': round(y8n_metrics.box.map75, 4),
        'precision': round(y8n_metrics.box.p.mean(), 4),
        'recall': round(y8n_metrics.box.r.mean(), 4),
        'duration_min': duration_min
    }
    y8n_results.append(y8n_result)

    with open(y8n_csv_filename, "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=y8n_result.keys())
        if trial == 0:
            writer.writeheader()
        writer.writerow(y8n_result)

    if current_map50 > y8n_best_map:
        y8n_best_map = current_map50
        y8n_best_params = y8n_params
        print(f"🔥 New best! mAP@0.5: {y8n_best_map:.4f}")

print(f"\n📝 All YOLOv8n results saved to {y8n_csv_filename}")
print("\n🏆 Best hyperparameters found:")
print(y8n_best_params)
print(f"Best mAP@0.5: {y8n_best_map:.4f}")



🔍 YOLOv8n Trial 1/5 | Params: {'lr0': 0.01526, 'batch': 32, 'weight_decay': 0.00066, 'momentum': 0.929, 'warmup_epochs': 3, 'patience': 20}
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cpu CPU (AMD Ryzen 5 7430U with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01526, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.929, mosaic=1.0, multi_scale=False, name=trial_1, nbs=64,

train: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\train\labels.cache... 161 images, 0 backgrounds, 0 corrupt: 100%|██████████| 161/161 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.00.0 ms, read: 120.248.2 MB/s, size: 13.3 KB)



c:\Users\muhdf\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\valid\labels.cache... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<?, ?it/s]

Plotting labels to runs\yolov8n_random_search\trial_1\labels.jpg... 


...


15 epochs completed in 0.032 hours.
Optimizer stripped from runs\yolov8n_random_search\trial_5\weights\last.pt, 6.2MB
Optimizer stripped from runs\yolov8n_random_search\trial_5\weights\best.pt, 6.2MB

Validating runs\yolov8n_random_search\trial_5\weights\best.pt...
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cpu CPU (AMD Ryzen 5 7430U with Radeon Graphics)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.52it/s]


                   all         39         95    0.00822      0.757      0.181      0.137
             BAD_APPLE          3         12    0.00701        0.5     0.0305     0.0232
         BAD_JACKFRUIT          4          9    0.00844      0.667      0.206      0.151
             BAD_MANGO          5          9    0.00481      0.444      0.132      0.098
            BAD_ORANGE          6          6    0.00913          1      0.672      0.577
        BAD_STRAWBERRY          4          7    0.00694      0.857      0.229      0.169
            GOOD_APPLE          6         17     0.0148      0.706      0.103     0.0565
        GOOD_JACKFRUIT          4          9    0.00925      0.889      0.198      0.115
            GOOD_MANGO          5         17     0.0126      0.647     0.0713     0.0489
           GOOD_ORANGE          2          2    0.00223          1      0.101     0.0738
       GOOD_STRAWBERRY          3          7      0.007      0.857     0.0711     0.0561
Speed: 0.3ms preproce

val: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\valid\labels.cache... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<?, ?it/s]
c:\Users\muhdf\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.70it/s]


                   all         39         95    0.00822      0.757      0.181      0.137
             BAD_APPLE          3         12    0.00701        0.5     0.0305     0.0232
         BAD_JACKFRUIT          4          9    0.00844      0.667      0.206      0.151
             BAD_MANGO          5          9    0.00481      0.444      0.132      0.098
            BAD_ORANGE          6          6    0.00913          1      0.672      0.577
        BAD_STRAWBERRY          4          7    0.00694      0.857      0.229      0.169
            GOOD_APPLE          6         17     0.0148      0.706      0.103     0.0565
        GOOD_JACKFRUIT          4          9    0.00925      0.889      0.198      0.115
            GOOD_MANGO          5         17     0.0126      0.647     0.0713     0.0489
           GOOD_ORANGE          2          2    0.00223          1      0.101     0.0738
       GOOD_STRAWBERRY          3          7      0.007      0.857     0.0711     0.0561
Speed: 0.3ms preproce

In [14]:
# Final Training Y8N with Best Hyperparameters
print("\nStarting final training with best parameters...")
y8n_final_model = YOLO("yolov8n.pt")
y8n_final_model.train(
    data="data.yaml",
    epochs=50,
    imgsz=256,
    name="y8n_final_training",
    project="runs/final_yolov8n",
    **y8n_best_params
)

# Final validation
metrics = y8n_final_model.val()
print("\nFinal Validation Metrics After Tuning:")
print(f"mAP@0.5:0.95    {metrics.box.map:.4f}")
print(f"mAP@0.5:        {metrics.box.map50:.4f}")
print(f"mAP@0.75:       {metrics.box.map75:.4f}")
print(f"Precision:      {metrics.box.p.mean():.4f}")
print(f"Recall:         {metrics.box.r.mean():.4f}")


Starting final training with best parameters...
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cpu CPU (AMD Ryzen 5 7430U with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00795, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.857, mosaic=1.0, multi_scale=False, name=y8n_final_training2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patie

train: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\train\labels.cache... 161 images, 0 backgrounds, 0 corrupt: 100%|██████████| 161/161 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.10.0 ms, read: 31.616.1 MB/s, size: 13.3 KB)



c:\Users\muhdf\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\valid\labels.cache... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<?, ?it/s]

Plotting labels to runs\final_yolov8n\y8n_final_training2\labels.jpg... 


...


50 epochs completed in 0.106 hours.
Optimizer stripped from runs\final_yolov8n\y8n_final_training2\weights\last.pt, 6.2MB
Optimizer stripped from runs\final_yolov8n\y8n_final_training2\weights\best.pt, 6.2MB

Validating runs\final_yolov8n\y8n_final_training2\weights\best.pt...
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cpu CPU (AMD Ryzen 5 7430U with Radeon Graphics)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.26it/s]


                   all         39         95      0.601      0.554      0.593      0.394
             BAD_APPLE          3         12      0.489      0.167      0.226      0.178
         BAD_JACKFRUIT          4          9      0.276      0.444      0.281      0.155
             BAD_MANGO          5          9      0.556      0.333      0.465      0.275
            BAD_ORANGE          6          6      0.708      0.833      0.859      0.678
        BAD_STRAWBERRY          4          7      0.458      0.429      0.574      0.447
            GOOD_APPLE          6         17      0.896      0.647      0.836      0.559
        GOOD_JACKFRUIT          4          9          1      0.189      0.515      0.307
            GOOD_MANGO          5         17       0.68      0.502      0.641      0.382
           GOOD_ORANGE          2          2      0.271          1      0.662      0.422
       GOOD_STRAWBERRY          3          7      0.679          1      0.872      0.538
Speed: 0.4ms preproce

val: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\valid\labels.cache... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<?, ?it/s]
c:\Users\muhdf\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.37it/s]


                   all         39         95      0.601      0.554      0.593      0.394
             BAD_APPLE          3         12      0.489      0.167      0.226      0.178
         BAD_JACKFRUIT          4          9      0.276      0.444      0.281      0.155
             BAD_MANGO          5          9      0.556      0.333      0.465      0.275
            BAD_ORANGE          6          6      0.708      0.833      0.859      0.678
        BAD_STRAWBERRY          4          7      0.458      0.429      0.574      0.447
            GOOD_APPLE          6         17      0.896      0.647      0.836      0.559
        GOOD_JACKFRUIT          4          9          1      0.189      0.515      0.307
            GOOD_MANGO          5         17       0.68      0.502      0.641      0.382
           GOOD_ORANGE          2          2      0.271          1      0.662      0.422
       GOOD_STRAWBERRY          3          7      0.679          1      0.872      0.538
Speed: 0.3ms preproce

---
### Random Search Hyper Parameter Tuning with YOLOv11n

In [9]:
from ultralytics import YOLO
import random
import csv
import os
import time

# Define parameter ranges
y11n_param_ranges = {
    'lr0': (0.0005, 0.02),
    'batch': [16, 32, 64],
    'weight_decay': (0.0001, 0.001),
    'momentum': (0.85, 0.98),
    'warmup_epochs': [2, 3, 5, 7],
    'patience': [10, 20, 30]
}

y11n_num_trials = 5
y11n_best_map = 0
y11n_best_params = {}
y11n_csv_filename = "yolov11n_random_search_results.csv"
y11n_results = []

# Optional: Remove old CSV
if os.path.exists(y11n_csv_filename):
    os.remove(y11n_csv_filename)

for trial in range(y11n_num_trials):
    # Unique randomness per trial
    random.seed(time.time() + trial)

    # Randomly sample hyperparameters
    y11n_params = {
        'lr0': round(random.uniform(*y11n_param_ranges['lr0']), 5),
        'batch': random.choice(y11n_param_ranges['batch']),
        'weight_decay': round(random.uniform(*y11n_param_ranges['weight_decay']), 5),
        'momentum': round(random.uniform(*y11n_param_ranges['momentum']), 3),
        'warmup_epochs': random.choice(y11n_param_ranges['warmup_epochs']),
        'patience': random.choice(y11n_param_ranges['patience'])
    }

    print(f"\n🔍 YOLOv11n Trial {trial+1}/{y11n_num_trials} | Params: {y11n_params}")

    start_time = time.time()

    # Train model
    y11n_model = YOLO("yolo11n.pt")
    y11n_model.train(
        data="data.yaml",
        epochs=15,
        imgsz=256,
        **y11n_params,
        name=f"trial_{trial+1}",
        project="runs/yolov11n_random_search",
    )

    # Validate model
    y11n_metrics = y11n_model.val()
    current_map50 = y11n_metrics.box.map50

    duration_min = round((time.time() - start_time) / 60, 2)

    print(f"\n📊 Trial {trial+1} Validation Metrics:")
    print(f"mAP@0.5:0.95    {y11n_metrics.box.map:.4f}")
    print(f"mAP@0.5:        {y11n_metrics.box.map50:.4f}")
    print(f"mAP@0.75:       {y11n_metrics.box.map75:.4f}")
    print(f"Precision:      {y11n_metrics.box.p.mean():.4f}")
    print(f"Recall:         {y11n_metrics.box.r.mean():.4f}")
    print(f"⏱️ Duration:     {duration_min:.2f} min")

    # Save result
    y11n_result = {
        'trial': trial + 1,
        **y11n_params,
        'map50': round(y11n_metrics.box.map50, 4),
        'map5095': round(y11n_metrics.box.map, 4),
        'map75': round(y11n_metrics.box.map75, 4),
        'precision': round(y11n_metrics.box.p.mean(), 4),
        'recall': round(y11n_metrics.box.r.mean(), 4),
        'duration_min': duration_min
    }
    y11n_results.append(y11n_result)

    # Save to CSV
    with open(y11n_csv_filename, "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=y11n_result.keys())
        if trial == 0:
            writer.writeheader()
        writer.writerow(y11n_result)

    # Update best
    if current_map50 > y11n_best_map:
        y11n_best_map = current_map50
        y11n_best_params = y11n_params
        print(f"🔥 New best! mAP@0.5: {y11n_best_map:.4f}")

# Final summary
print(f"\n📝 All YOLOv11n results saved to {y11n_csv_filename}")
print("\n🏆 Best hyperparameters found:")
print(y11n_best_params)
print(f"Best mAP@0.5: {y11n_best_map:.4f}")



🔍 YOLOv11n Trial 1/5 | Params: {'lr0': 0.01385, 'batch': 16, 'weight_decay': 0.00079, 'momentum': 0.953, 'warmup_epochs': 3, 'patience': 30}
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cpu CPU (AMD Ryzen 5 7430U with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01385, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.953, mosaic=1.0, multi_scale=False, name=trial_1, nbs=64

train: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\train\labels.cache... 161 images, 0 backgrounds, 0 corrupt: 100%|██████████| 161/161 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.30.0 ms, read: 30.415.8 MB/s, size: 13.3 KB)



c:\Users\muhdf\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\valid\labels.cache... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<?, ?it/s]

Plotting labels to runs\yolov11n_random_search\trial_1\labels.jpg... 


...


15 epochs completed in 0.035 hours.
Optimizer stripped from runs\yolov11n_random_search\trial_5\weights\last.pt, 5.4MB
Optimizer stripped from runs\yolov11n_random_search\trial_5\weights\best.pt, 5.4MB

Validating runs\yolov11n_random_search\trial_5\weights\best.pt...
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cpu CPU (AMD Ryzen 5 7430U with Radeon Graphics)
YOLO11n summary (fused): 100 layers, 2,584,102 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.52it/s]


                   all         39         95      0.333      0.422       0.41      0.282
             BAD_APPLE          3         12      0.225      0.167      0.258      0.178
         BAD_JACKFRUIT          4          9      0.122      0.222       0.12     0.0759
             BAD_MANGO          5          9      0.292      0.444      0.287      0.148
            BAD_ORANGE          6          6      0.499      0.167       0.32       0.27
        BAD_STRAWBERRY          4          7       0.07      0.143      0.116     0.0803
            GOOD_APPLE          6         17      0.382      0.294      0.272      0.175
        GOOD_JACKFRUIT          4          9      0.738      0.317      0.525      0.357
            GOOD_MANGO          5         17      0.575      0.471      0.593      0.312
           GOOD_ORANGE          2          2      0.114          1      0.995      0.796
       GOOD_STRAWBERRY          3          7       0.31          1      0.616      0.429
Speed: 0.4ms preproce

val: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\valid\labels.cache... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<?, ?it/s]
c:\Users\muhdf\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.41it/s]


                   all         39         95      0.333      0.422       0.41      0.282
             BAD_APPLE          3         12      0.225      0.167      0.258      0.178
         BAD_JACKFRUIT          4          9      0.122      0.222       0.12     0.0759
             BAD_MANGO          5          9      0.292      0.444      0.287      0.148
            BAD_ORANGE          6          6      0.499      0.167       0.32       0.27
        BAD_STRAWBERRY          4          7       0.07      0.143      0.116     0.0803
            GOOD_APPLE          6         17      0.382      0.294      0.272      0.175
        GOOD_JACKFRUIT          4          9      0.738      0.317      0.525      0.357
            GOOD_MANGO          5         17      0.575      0.471      0.593      0.312
           GOOD_ORANGE          2          2      0.114          1      0.995      0.796
       GOOD_STRAWBERRY          3          7       0.31          1      0.616      0.429
Speed: 0.4ms preproce

In [10]:
# Final training with best parameters
final_y11n = YOLO("yolo11n.pt")
final_y11n.train(
    data="data.yaml",
    epochs=50,
    imgsz=256,
    name="final_training",
    project="runs/final_yolo11n",
    **y11n_best_params
)

Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cpu CPU (AMD Ryzen 5 7430U with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01792, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.944, mosaic=1.0, multi_scale=False, name=final_training, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=30, perspective=0.0, plots=True, pose=12.0, pretra

train: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\train\labels.cache... 161 images, 0 backgrounds, 0 corrupt: 100%|██████████| 161/161 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.00.0 ms, read: 120.450.5 MB/s, size: 13.3 KB)



c:\Users\muhdf\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\valid\labels.cache... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<?, ?it/s]

Plotting labels to runs\final_yolo11n\final_training\labels.jpg... 


...

                   all         39         95       0.59      0.707      0.655      0.429



50 epochs completed in 0.103 hours.
Optimizer stripped from runs\final_yolo11n\final_training\weights\last.pt, 5.4MB
Optimizer stripped from runs\final_yolo11n\final_training\weights\best.pt, 5.4MB

Validating runs\final_yolo11n\final_training\weights\best.pt...
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cpu CPU (AMD Ryzen 5 7430U with Radeon Graphics)
YOLO11n summary (fused): 100 layers, 2,584,102 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.84it/s]


                   all         39         95      0.677      0.678      0.659      0.434
             BAD_APPLE          3         12      0.708      0.167      0.198      0.168
         BAD_JACKFRUIT          4          9      0.311      0.778      0.443      0.143
             BAD_MANGO          5          9          1      0.325      0.658      0.339
            BAD_ORANGE          6          6      0.852      0.965      0.948      0.796
        BAD_STRAWBERRY          4          7      0.867      0.857       0.93      0.666
            GOOD_APPLE          6         17      0.776      0.816      0.898      0.486
        GOOD_JACKFRUIT          4          9      0.726      0.222      0.718      0.506
            GOOD_MANGO          5         17      0.452      0.647      0.522      0.262
           GOOD_ORANGE          2          2      0.213          1      0.284      0.213
       GOOD_STRAWBERRY          3          7      0.864          1      0.995       0.76
Speed: 0.3ms preproce

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002523668EE50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,   

In [11]:
# Validate and print metrics
metrics = final_y11n.val()
current_map = metrics.box.map50
    
# Print all metrics in a clean format
print("\nFinal Validation Metrics After Tuning:")
print(f"mAP@0.5:0.95    {metrics.box.map:.4f}")
print(f"mAP@0.5:        {metrics.box.map50:.4f}")
print(f"mAP@0.75:       {metrics.box.map75:.4f}")
print(f"Precision:      {metrics.box.p.mean():.4f}")
print(f"Recall:         {metrics.box.r.mean():.4f}")

Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cpu CPU (AMD Ryzen 5 7430U with Radeon Graphics)
YOLO11n summary (fused): 100 layers, 2,584,102 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 75.544.6 MB/s, size: 9.9 KB)


val: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\valid\labels.cache... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<?, ?it/s]
c:\Users\muhdf\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.04it/s]


                   all         39         95      0.677      0.678      0.659      0.434
             BAD_APPLE          3         12      0.708      0.167      0.198      0.168
         BAD_JACKFRUIT          4          9      0.311      0.778      0.443      0.143
             BAD_MANGO          5          9          1      0.325      0.658      0.339
            BAD_ORANGE          6          6      0.852      0.965      0.948      0.796
        BAD_STRAWBERRY          4          7      0.867      0.857       0.93      0.666
            GOOD_APPLE          6         17      0.776      0.816      0.898      0.486
        GOOD_JACKFRUIT          4          9      0.726      0.222      0.718      0.506
            GOOD_MANGO          5         17      0.452      0.647      0.522      0.262
           GOOD_ORANGE          2          2      0.213          1      0.284      0.213
       GOOD_STRAWBERRY          3          7      0.864          1      0.995       0.76
Speed: 0.3ms preproce

---
YOLO8S

In [19]:
from ultralytics import YOLO
import random
import csv
import os
import time

# Parameter ranges for YOLOv8s
y8s_param_ranges = {
    'lr0': (0.0005, 0.02),
    'batch': [16, 32, 64],
    'weight_decay': (0.0001, 0.001),
    'momentum': (0.85, 0.98),
    'warmup_epochs': [2, 3, 5, 7],
    'patience': [10, 20, 30]
}

y8s_num_trials = 5
y8s_best_map = 0
y8s_best_params = {}
y8s_csv_filename = "yolov8s_random_search_results.csv"
y8s_results = []

if os.path.exists(y8s_csv_filename):
    os.remove(y8s_csv_filename)

for trial in range(y8s_num_trials):
    random.seed(time.time() + trial)

    y8s_params = {
        'lr0': round(random.uniform(*y8s_param_ranges['lr0']), 5),
        'batch': random.choice(y8s_param_ranges['batch']),
        'weight_decay': round(random.uniform(*y8s_param_ranges['weight_decay']), 5),
        'momentum': round(random.uniform(*y8s_param_ranges['momentum']), 3),
        'warmup_epochs': random.choice(y8s_param_ranges['warmup_epochs']),
        'patience': random.choice(y8s_param_ranges['patience'])
    }

    print(f"\n🔍 YOLOv8s Trial {trial+1}/{y8s_num_trials} | Params: {y8s_params}")

    start_time = time.time()

    y8s_model = YOLO("yolov8s.pt")
    y8s_model.train(
        data="data.yaml",
        epochs=15,
        imgsz=256,
        **y8s_params,
        name=f"trial_{trial+1}",
        project="runs/yolov8s_random_search",
    )

    y8s_metrics = y8s_model.val()
    current_map50 = y8s_metrics.box.map50
    duration_min = round((time.time() - start_time) / 60, 2)

    print(f"\n📊 Trial {trial+1} Validation Metrics:")
    print(f"mAP@0.5:0.95    {y8s_metrics.box.map:.4f}")
    print(f"mAP@0.5:        {y8s_metrics.box.map50:.4f}")
    print(f"mAP@0.75:       {y8s_metrics.box.map75:.4f}")
    print(f"Precision:      {y8s_metrics.box.p.mean():.4f}")
    print(f"Recall:         {y8s_metrics.box.r.mean():.4f}")
    print(f"⏱️ Duration:     {duration_min:.2f} min")

    y8s_result = {
        'trial': trial + 1,
        **y8s_params,
        'map50': round(y8s_metrics.box.map50, 4),
        'map5095': round(y8s_metrics.box.map, 4),
        'map75': round(y8s_metrics.box.map75, 4),
        'precision': round(y8s_metrics.box.p.mean(), 4),
        'recall': round(y8s_metrics.box.r.mean(), 4),
        'duration_min': duration_min
    }
    y8s_results.append(y8s_result)

    with open(y8s_csv_filename, "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=y8s_result.keys())
        if trial == 0:
            writer.writeheader()
        writer.writerow(y8s_result)

    if current_map50 > y8s_best_map:
        y8s_best_map = current_map50
        y8s_best_params = y8s_params
        print(f"🔥 New best! mAP@0.5: {y8s_best_map:.4f}")

print(f"\n📝 All YOLOv8s results saved to {y8s_csv_filename}")
print("\n🏆 Best hyperparameters found:")
print(y8s_best_params)
print(f"Best mAP@0.5: {y8s_best_map:.4f}")



🔍 YOLOv8s Trial 1/5 | Params: {'lr0': 0.01715, 'batch': 64, 'weight_decay': 0.00085, 'momentum': 0.872, 'warmup_epochs': 5, 'patience': 20}
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cpu CPU (AMD Ryzen 5 7430U with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01715, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.872, mosaic=1.0, multi_scale=False, name=trial_13, nbs=64

train: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\train\labels.cache... 161 images, 0 backgrounds, 0 corrupt: 100%|██████████| 161/161 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.10.0 ms, read: 23.610.4 MB/s, size: 13.3 KB)



c:\Users\muhdf\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\valid\labels.cache... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<?, ?it/s]

Plotting labels to runs\yolov8s_random_search\trial_13\labels.jpg... 


...


15 epochs completed in 0.134 hours.
Optimizer stripped from runs\yolov8s_random_search\trial_5\weights\last.pt, 22.5MB
Optimizer stripped from runs\yolov8s_random_search\trial_5\weights\best.pt, 22.5MB

Validating runs\yolov8s_random_search\trial_5\weights\best.pt...
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cpu CPU (AMD Ryzen 5 7430U with Radeon Graphics)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.89s/it]


                   all         39         95       0.47      0.519      0.482      0.339
             BAD_APPLE          3         12      0.155      0.167      0.238      0.181
         BAD_JACKFRUIT          4          9      0.304      0.333      0.319      0.202
             BAD_MANGO          5          9      0.786      0.333      0.419      0.299
            BAD_ORANGE          6          6      0.547      0.808      0.839      0.744
        BAD_STRAWBERRY          4          7      0.221      0.429      0.346      0.249
            GOOD_APPLE          6         17      0.645      0.588      0.667      0.352
        GOOD_JACKFRUIT          4          9      0.525      0.495      0.597      0.412
            GOOD_MANGO          5         17      0.487      0.176      0.295      0.148
           GOOD_ORANGE          2          2      0.147          1      0.182      0.136
       GOOD_STRAWBERRY          3          7      0.883      0.857      0.913      0.665
Speed: 0.3ms preproce

val: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\valid\labels.cache... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<?, ?it/s]
c:\Users\muhdf\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.87s/it]


                   all         39         95       0.47      0.519      0.482      0.339
             BAD_APPLE          3         12      0.155      0.167      0.238      0.181
         BAD_JACKFRUIT          4          9      0.304      0.333      0.319      0.202
             BAD_MANGO          5          9      0.786      0.333      0.419      0.299
            BAD_ORANGE          6          6      0.547      0.808      0.839      0.744
        BAD_STRAWBERRY          4          7      0.221      0.429      0.346      0.249
            GOOD_APPLE          6         17      0.645      0.588      0.667      0.352
        GOOD_JACKFRUIT          4          9      0.525      0.495      0.597      0.412
            GOOD_MANGO          5         17      0.487      0.176      0.295      0.148
           GOOD_ORANGE          2          2      0.147          1      0.182      0.136
       GOOD_STRAWBERRY          3          7      0.883      0.857      0.913      0.665
Speed: 0.3ms preproce

In [21]:
# Final Training Y8S with Best Hyperparameters
print("\nStarting final training with best parameters...")
y8s_final_model = YOLO("yolov8s.pt")
y8s_final_model.train(
    data="data.yaml",
    epochs=50,
    name="y8s_final_training",
    imgsz=256,
    project="runs/final_yolov8s",
    **y8s_best_params

)

# Final validation
metrics = y8s_final_model.val()
print("\nFinal Validation Metrics After Tuning:")
print(f"mAP@0.5:0.95    {metrics.box.map:.4f}")
print(f"mAP@0.5:        {metrics.box.map50:.4f}")
print(f"mAP@0.75:       {metrics.box.map75:.4f}")
print(f"Precision:      {metrics.box.p.mean():.4f}")
print(f"Recall:         {metrics.box.r.mean():.4f}")


Starting final training with best parameters...
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cpu CPU (AMD Ryzen 5 7430U with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00936, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.955, mosaic=1.0, multi_scale=False, name=y8s_final_training2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patie

train: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\train\labels.cache... 161 images, 0 backgrounds, 0 corrupt: 100%|██████████| 161/161 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.10.0 ms, read: 28.114.3 MB/s, size: 13.3 KB)



c:\Users\muhdf\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\valid\labels.cache... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<?, ?it/s]

Plotting labels to runs\final_yolov8s\y8s_final_training2\labels.jpg... 


...


27 epochs completed in 0.147 hours.
Optimizer stripped from runs\final_yolov8s\y8s_final_training2\weights\last.pt, 22.5MB
Optimizer stripped from runs\final_yolov8s\y8s_final_training2\weights\best.pt, 22.5MB

Validating runs\final_yolov8s\y8s_final_training2\weights\best.pt...
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cpu CPU (AMD Ryzen 5 7430U with Radeon Graphics)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.63s/it]


                   all         39         95      0.626      0.695      0.626      0.404
             BAD_APPLE          3         12      0.568      0.583      0.663       0.39
         BAD_JACKFRUIT          4          9      0.237      0.222      0.312      0.169
             BAD_MANGO          5          9       0.89      0.333      0.434      0.221
            BAD_ORANGE          6          6      0.829       0.81      0.924      0.769
        BAD_STRAWBERRY          4          7      0.504      0.857      0.749      0.529
            GOOD_APPLE          6         17       0.89      0.588      0.816      0.453
        GOOD_JACKFRUIT          4          9      0.616      0.556      0.412      0.237
            GOOD_MANGO          5         17      0.474          1      0.628      0.336
           GOOD_ORANGE          2          2       0.33          1      0.332      0.249
       GOOD_STRAWBERRY          3          7      0.919          1      0.995      0.688
Speed: 0.3ms preproce

val: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\valid\labels.cache... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<?, ?it/s]
c:\Users\muhdf\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.03it/s]


                   all         39         95      0.626      0.695      0.626      0.404
             BAD_APPLE          3         12      0.568      0.583      0.663       0.39
         BAD_JACKFRUIT          4          9      0.237      0.222      0.312      0.169
             BAD_MANGO          5          9       0.89      0.333      0.434      0.221
            BAD_ORANGE          6          6      0.829       0.81      0.924      0.769
        BAD_STRAWBERRY          4          7      0.504      0.857      0.749      0.529
            GOOD_APPLE          6         17       0.89      0.588      0.816      0.453
        GOOD_JACKFRUIT          4          9      0.616      0.556      0.412      0.237
            GOOD_MANGO          5         17      0.474          1      0.628      0.336
           GOOD_ORANGE          2          2       0.33          1      0.332      0.249
       GOOD_STRAWBERRY          3          7      0.919          1      0.995      0.688
Speed: 0.8ms preproce

---
### RESULTS

In [22]:
import pandas as pd


y11n_final_metrics = final_y11n.val()
y8n_final_metrics = y8n_final_model.val()
y8s_final_metrics = y8n_final_model.val()

results_summary = [
    {
        "Model": "YOLOv11n",
        "mAP@0.5:0.95": y11n_final_metrics.box.map,
        "mAP@0.5": y11n_final_metrics.box.map50,
        "mAP@0.75": y11n_final_metrics.box.map75,
        "Precision": y11n_final_metrics.box.p.mean(),
        "Recall": y11n_final_metrics.box.r.mean()
    },
    {
        "Model": "YOLOv8n",
        "mAP@0.5:0.95": y8n_final_metrics.box.map,
        "mAP@0.5": y8n_final_metrics.box.map50,
        "mAP@0.75": y8n_final_metrics.box.map75,
        "Precision": y8n_final_metrics.box.p.mean(),
        "Recall": y8n_final_metrics.box.r.mean()
    },
    {
        "Model": "YOLOv8s",
        "mAP@0.5:0.95": y8s_final_metrics.box.map,
        "mAP@0.5": y8s_final_metrics.box.map50,
        "mAP@0.75": y8s_final_metrics.box.map75,
        "Precision": y8s_final_metrics.box.p.mean(),
        "Recall": y8s_final_metrics.box.r.mean()
    }
]

df_summary = pd.DataFrame(results_summary)
print("\n📊 Model Comparison Table:")
print(df_summary.round(4))


Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cpu CPU (AMD Ryzen 5 7430U with Radeon Graphics)
val: Fast image access  (ping: 0.00.0 ms, read: 24.312.0 MB/s, size: 10.8 KB)


val: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\valid\labels.cache... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<?, ?it/s]
c:\Users\muhdf\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.43it/s]


                   all         39         95      0.677      0.678      0.659      0.434
             BAD_APPLE          3         12      0.708      0.167      0.198      0.168
         BAD_JACKFRUIT          4          9      0.311      0.778      0.443      0.143
             BAD_MANGO          5          9          1      0.325      0.658      0.339
            BAD_ORANGE          6          6      0.852      0.965      0.948      0.796
        BAD_STRAWBERRY          4          7      0.867      0.857       0.93      0.666
            GOOD_APPLE          6         17      0.776      0.816      0.898      0.486
        GOOD_JACKFRUIT          4          9      0.726      0.222      0.718      0.506
            GOOD_MANGO          5         17      0.452      0.647      0.522      0.262
           GOOD_ORANGE          2          2      0.213          1      0.284      0.213
       GOOD_STRAWBERRY          3          7      0.864          1      0.995       0.76
Speed: 1.3ms preproce

val: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\valid\labels.cache... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<?, ?it/s]
c:\Users\muhdf\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.61it/s]


                   all         39         95      0.601      0.554      0.593      0.394
             BAD_APPLE          3         12      0.489      0.167      0.226      0.178
         BAD_JACKFRUIT          4          9      0.276      0.444      0.281      0.155
             BAD_MANGO          5          9      0.556      0.333      0.465      0.275
            BAD_ORANGE          6          6      0.708      0.833      0.859      0.678
        BAD_STRAWBERRY          4          7      0.458      0.429      0.574      0.447
            GOOD_APPLE          6         17      0.896      0.647      0.836      0.559
        GOOD_JACKFRUIT          4          9          1      0.189      0.515      0.307
            GOOD_MANGO          5         17       0.68      0.502      0.641      0.382
           GOOD_ORANGE          2          2      0.271          1      0.662      0.422
       GOOD_STRAWBERRY          3          7      0.679          1      0.872      0.538
Speed: 0.3ms preproce

val: Scanning C:\Users\muhdf\OneDrive\Desktop\GALEYA\ML\FRUIT\valid\labels.cache... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<?, ?it/s]
c:\Users\muhdf\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.40it/s]


                   all         39         95      0.601      0.554      0.593      0.394
             BAD_APPLE          3         12      0.489      0.167      0.226      0.178
         BAD_JACKFRUIT          4          9      0.276      0.444      0.281      0.155
             BAD_MANGO          5          9      0.556      0.333      0.465      0.275
            BAD_ORANGE          6          6      0.708      0.833      0.859      0.678
        BAD_STRAWBERRY          4          7      0.458      0.429      0.574      0.447
            GOOD_APPLE          6         17      0.896      0.647      0.836      0.559
        GOOD_JACKFRUIT          4          9          1      0.189      0.515      0.307
            GOOD_MANGO          5         17       0.68      0.502      0.641      0.382
           GOOD_ORANGE          2          2      0.271          1      0.662      0.422
       GOOD_STRAWBERRY          3          7      0.679          1      0.872      0.538
Speed: 0.5ms preproce

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Hyperparameter data
param_data = [
    {"Model": "YOLOv8s", "lr0": 0.00936, "batch": 32, "weight_decay": 0.00028, "momentum": 0.955, "warmup_epochs": 3, "patience": 10, "Best mAP@0.5": 0.5663},
    {"Model": "YOLOv11n", "lr0": 0.01792, "batch": 16, "weight_decay": 0.00023, "momentum": 0.944, "warmup_epochs": 2, "patience": 30, "Best mAP@0.5": 0.4312},
    {"Model": "YOLOv8n", "lr0": 0.00795, "batch": 16, "weight_decay": 0.00089, "momentum": 0.857, "warmup_epochs": 5, "patience": 10, "Best mAP@0.5": 0.4297},
]

df_params = pd.DataFrame(param_data)

# Display table
print("📋 Best Hyperparameters Per Model:\n")
display(df_params)

# Plot comparison of hyperparameters (excluding categorical values)
hyperparams_only = df_params.drop(columns=["Model", "Best mAP@0.5"])
df_plot = pd.melt(df_params, id_vars=["Model"], value_vars=hyperparams_only.columns)

plt.figure(figsize=(12, 6))
sns.barplot(data=df_plot, x="variable", y="value", hue="Model")
plt.title("Hyperparameter Comparison Across YOLO Models")
plt.ylabel("Value")
plt.xlabel("Hyperparameter")
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()


📋 Best Hyperparameters Per Model:



,Model,lr0,batch,weight_decay,momentum,warmup_epochs,patience,Best mAP@0.5
0,YOLOv8s,0.00936,32,0.00028,0.955,3,10,0.5663
1,YOLOv11n,0.01792,16,0.00023,0.944,2,30,0.4312
2,YOLOv8n,0.00795,16,0.00089,0.857,5,10,0.4297


<Figure size 1200x600 with 1 Axes>